In [51]:
def forward_kinematics(theta1, theta2, l1, l2):
    x = l1*math.cos(theta1) + l2*math.cos(theta1 + theta2)
    y = l1*math.sin(theta1) + l2*math.sin(theta1 + theta2)
    return [x, y]

def inverse_kinematics(x, y, l1, l2):
    a = 2*x*l2
    b = 2*y*l2
    c =  l1*l1 - x*x - y*y  - l2*l2 
    psi = math.atan2(b, a)
    theta12 = psi + math.acos(-c/math.sqrt(a*a + b*b))
    theta1 = math.atan2((y - l2*math.sin(theta12))/l1, (x - l2*math.cos(theta12))/l1)
    return [theta1, theta12-theta1]

In [48]:
import math
import numpy

[x, y] = forward_kinematics(math.pi/2, 0, 1, 2)

[theta1, theta2] = inverse_kinematics(0, -1, 1, 2)

In [12]:
print(x,y)

1.8369701987210297e-16 3.0


In [49]:
print(theta1, theta2)

1.5707963267948968 -3.141592653589793


In [67]:
theta1 = numpy.random.rand()
theta2 = numpy.random.rand()
print(theta1, theta2)

l1 = 1
l2 = 2

[x, y] = forward_kinematics(theta1, theta2, l1, l2)
[t1, t2] = inverse_kinematics(x, y, l1, l2)

print(t1 - theta1, t2 - theta2)


0.6194169893868526 0.07765851659097589
-2.220446049250313e-15 3.3306690738754696e-15
